In [1]:
pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00


In [2]:
pip install finvader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [3]:
pip install yfinance

In [4]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.2 MB/s eta 0:00:00


In [ ]:
#url = "https://elite.finviz.com/export.ashx?v=150&p=i1&f=cap_0.01to,geo_usa|china|france|europe|australia|belgium|canada|chinahongkong|germany|hongkong|iceland|japan|newzealand|ireland|netherlands|norway|singapore|southkorea|sweden|taiwan|unitedarabemirates|unitedkingdom|switzerland|spain,sh_curvol_o1000,sh_price_u50,sh_relvol_o5,ta_change_u&ft=4&o=-change&ar=10&auth=40f88c6a-1d17-4359-8afc-16412534cd7a"

In [6]:
# Import necessary libraries
import requests
import pandas as pd
from finvizfinance.quote import finvizfinance
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from finvader import finvader
import yfinance as yf
import time
from requests.exceptions import HTTPError
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import random

# List of User-Agent strings
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; Pixel 3 XL Build/QP1A.190711.020) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Gecko/20100101 Firefox/85.0',
]

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Retry mechanism for requests
def fetch_with_retries(url, retries=5, backoff_factor=1):
    for attempt in range(retries):
        headers = {
            'User-Agent': random.choice(user_agents)  # Randomly select a User-Agent
        }
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response
        except HTTPError as e:
            print(f"HTTPError: {e} - Attempt {attempt + 1} of {retries}")
        except Exception as e:
            print(f"Error fetching {url}: {e} - Attempt {attempt + 1} of {retries}")

        wait_time = backoff_factor * (2 ** attempt)
        print(f"Waiting for {wait_time} seconds before retrying...")
        time.sleep(wait_time)
    return None

# Step 1: Download the data
url = "https://elite.finviz.com/export.ashx?v=150&p=i1&f=cap_0.01to,geo_usa|china|france|europe|australia|belgium|canada|chinahongkong|germany|hongkong|iceland|japan|newzealand|ireland|netherlands|norway|singapore|southkorea|sweden|taiwan|unitedarabemirates|unitedkingdom|switzerland|spain,sh_curvol_o1000,sh_price_u50,sh_relvol_o5,ta_change_u&ft=4&o=-change&ar=10&auth=6331d996-c388-4164-b12a-a1af717c75b4"
response = fetch_with_retries(url)
if response is not None:
    # Print the content to check if it's valid CSV
    print(response.text[:500])  # Print the first 500 characters of the response
    csv_path = "finviz.csv"
    with open(csv_path, "wb") as file:
        file.write(response.content)
else:
    raise Exception("Failed to download the CSV file.")


# Step 2: Read the CSV
df = pd.read_csv(csv_path)

# Step 3: Get news for each stock
news_list = []
current_date = datetime.now().strftime("%b-%d-%y")

for ticker in df['Ticker']:
    stock = finvizfinance(ticker)

    try:
        news_df = stock.ticker_news()

        # Check if news_df is None
        if news_df is None or news_df.empty:
            print(f"No news available for {ticker}. Skipping.")
            continue  # Skip to the next ticker if no news is found

        news_df['Ticker'] = ticker  # Add ticker symbol to the news DataFrame

        # Convert the Date field to datetime including time
        news_df['Date'] = pd.to_datetime(news_df['Date'], format='%b-%d-%y %I:%M%p')

        # Extract time from the Date column
        news_df['Time'] = news_df['Date'].dt.time

        # Filter the news by current date
        today_news_df = news_df[news_df['Date'].dt.strftime('%b-%d-%y') == current_date]

        if not today_news_df.empty:
            news_list.append(today_news_df)

    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")
        continue  # Continue to the next ticker if an error occurs

# Check if there is any news collected
if news_list:
    # Concatenate all today's news DataFrames
    all_today_news_df = pd.concat(news_list, ignore_index=True)
else:
    print("No news collected for any of the tickers.")



# Step 4: Scrape article content
def scrape_article(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        article_text = ' '.join([p.get_text() for p in paragraphs])
        return article_text
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

all_today_news_df['Article'] = all_today_news_df['Link'].apply(scrape_article)

# Preprocess the text using NLTK
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

all_today_news_df['Prep_Artcl'] = all_today_news_df['Article'].apply(preprocess_text)

# Initialize FinBERT sentiment analyzer and tokenizer
model_name = 'yiyanghkust/finbert-tone'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def analyze_sentiment(text):
    try:
        tokens = tokenizer(text, truncation=True, max_length=512, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**tokens)
            logits = outputs.logits
            probabilities = F.softmax(logits, dim=1).squeeze().tolist()
        labels = model.config.id2label
        scores = {labels[i]: probabilities[i] for i in range(len(labels))}
        return scores
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return {}

# Apply sentiment analysis on original and preprocessed articles
all_today_news_df['Sent_Orig_Scrs'] = all_today_news_df['Article'].apply(analyze_sentiment)
all_today_news_df['Sent_Prep_Scrs'] = all_today_news_df['Prep_Artcl'].apply(analyze_sentiment)

# Apply sentiment analysis using FinVADER
def analyze_sentiment_finvader(text):
    try:
        return finvader(text, use_sentibignomics=True, use_henry=True, indicator='compound')
    except Exception as e:
        print(f"Error analyzing sentiment with FinVADER: {e}")
        return {}

all_today_news_df['Sent_FinV'] = all_today_news_df['Article'].apply(analyze_sentiment_finvader)

# Determine the best predicted sentiment
def determine_sentiment(scores):
    return max(scores, key=scores.get)

# Add the best predicted sentiment columns
all_today_news_df['Sent_FinB'] = all_today_news_df['Sent_Orig_Scrs'].apply(determine_sentiment)
all_today_news_df['Sent_Prep'] = all_today_news_df['Sent_Prep_Scrs'].apply(determine_sentiment)
all_today_news_df['Sent_FinV'] = all_today_news_df['Sent_FinV'].apply(lambda x: 'Positive' if x > 0.05 else ('Negative' if x < -0.05 else 'Neutral'))

# Compute overall sentiment for each stock
def overall_sentiment(row):
    sentiments = [row['Sent_FinB'], row['Sent_Prep'], row['Sent_FinV']]
    positive_count = sentiments.count('Positive')
    negative_count = sentiments.count('Negative')
    neutral_count = sentiments.count('Neutral')
    if positive_count > max(negative_count, neutral_count):
        return 'Positive'
    elif negative_count > max(positive_count, neutral_count):
        return 'Negative'
    else:
        return 'Neutral'

all_today_news_df['Overall_Sent'] = all_today_news_df.apply(overall_sentiment, axis=1)

# Flatten the sentiment scores into separate columns
def flatten_sentiment_scores(df, column_name):
    sentiment_df = pd.json_normalize(df[column_name])
    sentiment_df.columns = [f"{column_name}_{col}" for col in sentiment_df.columns]
    return sentiment_df

sentiment_original_df = flatten_sentiment_scores(all_today_news_df, 'Sent_Orig_Scrs')
sentiment_preprocessed_df = flatten_sentiment_scores(all_today_news_df, 'Sent_Prep_Scrs')

# Concatenate the flattened sentiment scores with the original DataFrame
all_today_news_df = pd.concat([all_today_news_df, sentiment_original_df, sentiment_preprocessed_df], axis=1)

# Drop unnecessary columns
final_df = all_today_news_df.drop(columns=['Article', 'Prep_Artcl', 'Link', 'Sent_Orig_Scrs', 'Sent_Prep_Scrs'])

# Step 5: Predict future stock returns based on sentiment
def get_historical_data(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print(f"Error fetching historical data for {ticker}: {e}")
        return None

historical_data = {}
start_date = datetime.now() - timedelta(days=365)
end_date = datetime.now()

for ticker in final_df['Ticker'].unique():
    historical_data[ticker] = get_historical_data(ticker, start_date, end_date)

# Step 6: Calculate daily returns and price changes
for ticker, hist_df in historical_data.items():
    hist_df['Return'] = hist_df['Adj Close'].pct_change()

price_changes = []
for ticker in final_df['Ticker'].unique():
    hist_df = historical_data.get(ticker)
    if hist_df is not None:
        hist_df.reset_index(inplace=True)
        hist_df['Date'] = pd.to_datetime(hist_df['Date'])

        current_day_data = hist_df.loc[hist_df['Date'].dt.date == datetime.now().date()]
        if not current_day_data.empty:
            current_open_price = current_day_data.iloc[0]['Open']
            current_close_price = current_day_data.iloc[0]['Close']
            price_change = current_close_price - current_open_price
            price_changes.append({'Ticker': ticker, 'Price_Change': price_change})
        else:
            print(f"No current day data for {ticker}.")
    else:
        print(f"No historical data for {ticker}.")

price_change_df = pd.DataFrame(price_changes)
print(price_change_df)  # Check the content of price_change_df

final_df = final_df.merge(price_change_df, on='Ticker', how='left')

# Merge sentiment data with historical data
def merge_sentiment_with_returns(sentiment_df, historical_data):
    combined_data = []
    for ticker, hist_df in historical_data.items():
        ticker_sentiment = sentiment_df[sentiment_df['Ticker'] == ticker]
        ticker_sentiment['Date'] = pd.to_datetime(ticker_sentiment['Date']).dt.date
        hist_df = hist_df.reset_index()
        hist_df['Date'] = hist_df['Date'].dt.date
        merged_df = pd.merge(ticker_sentiment, hist_df, on='Date', how='inner')
        combined_data.append(merged_df)
    return pd.concat(combined_data, ignore_index=True)

combined_df = merge_sentiment_with_returns(final_df, historical_data)

# Prepare the data for model training
features = combined_df.drop(columns=['Ticker', 'Date', 'Return', 'Adj Close', 'Open', 'High', 'Low', 'Close', 'Volume'])
target = combined_df['Return']

# Ensure features contain only numeric columns
features = features.select_dtypes(include=['float64', 'int64'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train multiple regression models and evaluate them
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42, verbose=0)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    results[name] = {'MSE': mse, 'R2': r2, 'MAE': mae}
    print(f"{name} - MSE: {mse}, R2: {r2}, MAE: {mae}")

# Train and evaluate a neural network model
model_nn = Sequential()
model_nn.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model_nn.add(Dense(64, activation='relu'))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dense(1))

model_nn.compile(loss='mse', optimizer='adam')

model_nn.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

y_pred_nn = model_nn.predict(X_test)
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
mae_nn = mean_absolute_error(y_test, y_pred_nn)

# Add neural network results to the results dictionary
results['Neural Network'] = {'MSE': mse_nn, 'R2': r2_nn, 'MAE': mae_nn}

print(f"Neural Network - MSE: {mse_nn}, R2: {r2_nn}, MAE: {mae_nn}")

# Display all results
for name, metrics in results.items():
    print(f"{name} - MSE: {metrics['MSE']}, R2: {metrics['R2']}, MAE: {metrics['MAE']}")

# Find the best model based on the lowest Mean Squared Error (MSE)
best_model_name = min(results, key=lambda k: results[k]['MSE'])

# Define the best model
best_model = models.get(best_model_name)
if best_model_name == 'Neural Network':
    best_model = model_nn

# Using the best model for predictions
combined_df['Pred_Rtn'] = best_model.predict(features)

# Define thresholds for buy recommendations
buy_threshold = 0.02
high_return_threshold = 0.2

# Make recommendations based on predicted returns and overall sentiment
combined_df['Recom'] = combined_df.apply(
    lambda row: 'Buy' if (row['Pred_Rtn'] > high_return_threshold and row['Overall_Sent'] == 'Neutral')
                  or (row['Pred_Rtn'] > buy_threshold and row['Overall_Sent'] == 'Positive')
    else ('Sell' if row['Pred_Rtn'] < -buy_threshold and row['Overall_Sent'] == 'Negative'
          else 'Hold'), axis=1)


# Sort the DataFrame based on Date and Time
combined_df.sort_values(by=['Date', 'Time'], inplace=True)

# Save the final DataFrame to a CSV file
combined_df.to_csv("stock_news_today_with_sentiment_and_recommendations.csv", index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


"No.","Ticker","Company","Sector","Industry","Country","Market Cap","P/E","Volume","Price","Change"
1,"NUZE","Nuzee Inc","Consumer Defensive","Packaged Foods","USA",16.04,,20863470,6.71,81.35%
2,"NCNC","noco-noco Inc","Consumer Cyclical","Auto Parts","Singapore",31.95,3.62,88687298,0.24,54.18%
3,"LSH","Lakeside Holding Ltd.","Industrials","Integrated Freight & Logistics","USA",25.50,97.98,15018592,3.40,43.46%
4,"BFI","BurgerFi International Inc","Consumer Cyclical","Restaurants","USA",19.17,
Error fetching news for PITA: 'NoneType' object has no attribute 'find_all'


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Gradient Boosting - MSE: 0.0020575229567210444, R2: nan, MAE: 0.04535992677155734
XGBoost - MSE: 0.00038353572610171045, R2: nan, MAE: 0.019584068170370283
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 3, number of used features: 0
[LightGBM] [Info] Start training from score 0.187361
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


CatBoost - MSE: 0.000918425997594297, R2: nan, MAE: 0.03030554400756233
Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 6.0695
Epoch 2/50
1/1 [==============================] - 0s 18ms/step - loss: 20.1497
Epoch 3/50
1/1 [==============================] - 0s 20ms/step - loss: 4.4247
Epoch 4/50
1/1 [==============================] - 0s 14ms/step - loss: 1.7749
Epoch 5/50
1/1 [==============================] - 0s 13ms/step - loss: 8.9975
Epoch 6/50
1/1 [==============================] - 0s 24ms/step - loss: 6.4753
Epoch 7/50
1/1 [==============================] - 0s 33ms/step - loss: 0.8057
Epoch 8/50
1/1 [==============================] - 0s 13ms/step - loss: 1.0534
Epoch 9/50
1/1 [==============================] - 0s 13ms/step - loss: 4.6621
Epoch 10/50
1/1 [==============================] - 0s 14ms/step - loss: 4.5637
Epoch 11/50
1/1 [==============================] - 0s 17ms/step - loss: 1.5542
Epoch 12/50
1/1 [==============================] - 0s 15ms/step - los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
